In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from gensim import corpora
from gensim.models import LdaModel

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
documents = [
"Rafael Nadal Joins Roger Federer in Missing U.S. Open",
"Rafael Nadal Is Out of the Australian Open",
"Biden Announces Virus Measures",
"Biden's Virus Plans Meet Reality",
"Where Biden's Virus Plan Stands"
]

In [6]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]
preprocessed_documents

[['rafael', 'nadal', 'join', 'roger', 'federer', 'missing', 'open'],
 ['rafael', 'nadal', 'australian', 'open'],
 ['biden', 'announces', 'virus', 'measure'],
 ['biden', 'virus', 'plan', 'meet', 'reality'],
 ['biden', 'virus', 'plan', 'stand']]

In [7]:
dictionary = corpora.Dictionary(preprocessed_documents)

corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]

In [8]:
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)

In [10]:
article_labels = []

for i, doc in enumerate(preprocessed_documents):
    bow = dictionary.doc2bow(doc)
    topics = lda_model.get_document_topics(bow)
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    article_labels.append(dominant_topic)

In [12]:
import pandas as pd
df = pd.DataFrame({"Article": documents, "Topic": article_labels})

print("Table with Articles and Topic:")
print(df)
print()

Table with Articles and Topic:
                                             Article  Topic
0  Rafael Nadal Joins Roger Federer in Missing U....      1
1         Rafael Nadal Is Out of the Australian Open      1
2                     Biden Announces Virus Measures      0
3                   Biden's Virus Plans Meet Reality      0
4                    Where Biden's Virus Plan Stands      0



In [14]:
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"-{word.strip()} (weight: {weight.strip()})")
    print()

Top Terms for Each Topic:
Topic 0:
-"biden" (weight: 0.166)
-"virus" (weight: 0.166)
-"plan" (weight: 0.119)
-"reality" (weight: 0.071)
-"meet" (weight: 0.071)
-"measure" (weight: 0.071)
-"announces" (weight: 0.071)
-"stand" (weight: 0.071)
-"open" (weight: 0.024)
-"australian" (weight: 0.024)

Topic 1:
-"nadal" (weight: 0.131)
-"rafael" (weight: 0.131)
-"open" (weight: 0.131)
-"federer" (weight: 0.078)
-"missing" (weight: 0.078)
-"join" (weight: 0.078)
-"roger" (weight: 0.078)
-"australian" (weight: 0.078)
-"virus" (weight: 0.027)
-"biden" (weight: 0.027)

